# 4 - 词向量和fasttext
在词袋模型和n-gram模型中，词都是由一个整数索引表示的。然而，就像我们在上一节提到的，这种做法使得每个词都是平等而孤立的个体，抹去了自然语言中词语先后顺序与距离的意义。n-gram利用窗口考察上下文，为的就是还原顺序与距离所包含的信息。

本节介绍一种更加本质的方法：将词用**向量**表示。通过某种手段训练向量模型，使得意思相近的词向量距离较小。

以下是几个核心概念：
+ **one-hot representaion**：词向量的独热表示
+ **distributed representation**：词向量的分布式表示
+ **embedding**：嵌入
+ **word2vec**

## One-Hot Representation
独热表示法由**索引**直接得到，本质上与其没有区别。

如：`"enterprise"` -> `index:3` -> `one-hot:(0,0,1,0,0,0,0,0,0,0)`\[words_num=10\]

也就是说，词向量的维数=词典的大小。在实际应用中，我们不将独热向量作为主模型（分类/翻译...）的输入，而是在其基础上训练得到分布式表示。这个过程即为**嵌入(embedding)**。

## Distributed Representation
首先说为什么叫distributed，这需要和one-hot表示区分来看。在独热表示中，词向量所包含的信息就全部集中在其特有的那一个“1”上，那么除此以外的所有位对该词向量而言就是纯粹的浪费。基于这个思想，分布式表示就是要让信息分散开来，即向量中的每一位数字都是有意义的，如此一来也节省了空间。

在2013Mikolov提出了两种训练分布式词向量的模型：**CBOW、skip-gram**，之后被收录在word2vec工具中。原论文[见此](https://arxiv.org/pdf/1301.3781v3.pdf)，推荐阅读Xin Rong后来写的[说明论文](https://arxiv.org/pdf/1411.2738.pdf)，其对参数更新的具体过程、后来的优化方法（哈夫曼树层级结构、负采样）都作了详细的解释。

## word2vec
是一个工具包，收录了CBOW、skip-gram模型。

概念区分（**其他词嵌入工具**）：
+ **GloVe**：Global Vectors for Word Representation，基于全局词频统计的词表征模型。同样是利用n-gram的共现思想，glove一次考察两个词的共现概率，即n=2，这样可以有效改善word2vec模型中窗口内部词序意义被抹去的问题。但是据说两种工具在各种语料库的表现上各有千秋，总的来说没有太大差异。GloVe方法的解读可参考[这篇文章](http://www.fanyeong.com/2018/02/19/glove-in-detail/)。
+ **fasttext**：也是由Mikolov在16年提出的，主要针对**分类问题**。其使用的模型框架和word2vec很相似，不同之处在于：

  (具体可以参考机器之心的[这篇文章](https://www.jiqizhixin.com/articles/2018-06-05-3))
  1. 其将n-gram思想进一步应用在词上，即将单词拆分为定长的n-gram(subgram)，作为模型的输入。由于西方语系中多有词缀的概念，这种做法很好地解决了word2vec模型无法处理生词以及低频词得不到足够训练的问题。
  2. 从输入到隐层不再训练一个`words_num*words_dim`的参数矩阵，而是直接线性求平均，大大提高了训练速度。
  3. 训练过程中的输出不再是中间词，而是该句的标签。词向量为分类问题的附属输出。

### 使用

word2vec - [gensim](https://radimrehurek.com/gensim/models/word2vec.html), 
GloVe - [glove](https://nlp.stanford.edu/projects/glove/), 
fasttext - [fasttext](https://fasttext.cc/)

下面展示word2vec和fasttext的使用。

(注意：词表征模型的训练需要**大量语料**，我所使用的TOS台词肯定是不够的，此处仅供演示)


In [2]:
import os
import re
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

import fasttext

#### word2vec
工具包中有已经训练好的模型，可以直接下载使用。

使用`字典`方法获取词向量，利用`model.wv.most_similar(word)`查找意思相近的词。

另外，当词汇量很大时，可以创建字典将most_similar_words作为值进行存储（`cache`）。

In [ ]:
google_wv_model = api.load('word2vec-google-news-300')

[--------------------------------------------------] 0.0% 0.0/1662.8MB downloaded

In [ ]:
# print("bridge:",google_wv_model['bridge']) #查看词向量
print("most similar words:\n",google_wv_model.wv.most_similar('bridge'))

# cache
# most_similars_precalc = {word : google_wv_model.wv.most_similar(word) for word in google_wv_model.wv.index2word}

但一般对于具体问题，最好用手上的语料库训练模型，才能更好地挖掘文本信息。比如说“bridge”在TOS台词中的意义多为“舰桥”（指挥室），而非我们常说的“桥”。利用pre-trained模型得到的结果很难是我们想要的。

以下为自定义方法：

In [3]:
# 预处理，此处无需除去停词
def preprocess(text):
    words = re.split(r',|\.|\?|!|"|\'| |\(|\)',text.lower())
    words = list(filter(lambda x:len(x)>1,words))
    return words

# 通过定义迭代器向模型输入数据
class MyCorpus:
    def __iter__(self):
        # 已提前将所有的台词按句分行整合进文档"processed_data/lines_corpus.txt"，并除去角色和场景信息
        for line in open('processed_data/lines_corpus.txt','r').read().splitlines():
            if len(set(line))<=2: continue
            yield preprocess(line)

可指定模型`Word2Vec()`的**训练参数**：
+ min_count：除去低频词，默认为5
+ size：词向量的维数，默认为100。据称200最佳
+ workers：用于加速，只有当装了Cython才有用。不然会受制于GIL，同时训练会"miserably slow"(官方原话)

In [4]:
# 迭代器
sentences = MyCorpus()
# 自定义模型
my_wv_model = Word2Vec(sentences=sentences)

In [5]:
print("word2vec - most similar words:\n",my_wv_model.wv.most_similar('bridge'))

word2vec - most similar words:
 [('room', 0.9295594692230225), ('engineering', 0.919005274772644), ('transporter', 0.9140746593475342), ('guard', 0.911492109298706), ('report', 0.9017926454544067), ('communicator', 0.9010648131370544), ('check', 0.8947445154190063), ('promptly', 0.8887509107589722), ('screen', 0.8880694508552551), ('whisper', 0.8873355388641357)]


---
#### fasttext
fasttext使用起来非常方便，可以直接在命令行操作，也可以利用python的接口。

其提供两种任务处理：[文本分类问题](https://fasttext.cc/docs/en/supervised-tutorial.html)和[训练词表征模型](https://fasttext.cc/docs/en/unsupervised-tutorial.html)。

1. 分类问题

使用`train_supervised`方法。

可以指定的**训练参数**有：epoch（迭代数）、lr（学习率）和`wordNgrams`（模型训练中考察共现的窗口大小，默认为3）。当处理对语序非常敏感的情绪文本是，将wordNgrams设置为2能有效提升模型准确率（相当于进一步考虑语序）

In [6]:
fast_class_model = fasttext.train_supervised('processed_data/best3_classification.train')
print(fast_class_model.test('processed_data/best3_classification.test'))

(3592, 0.669543429844098, 0.669543429844098)


三个数字分别是**样本数、precision、recall**，其中对于单标签问题precision、recall的值是一样的。

fasttext的分类模型训练虽然极其简单，但是对于**输入数据格式**有所要求 -> 每一行表示一个样本，标签和文本用空格隔开，并在标签前后加上双下划线

如： `__SPOCK__ Sensor scan to one half parsec. Negative, Captain.`

同时对数据进行`预处理`（去标点、忽略大小）有利于提高模型准确度。

In [7]:
print(fast_class_model.predict("Beam me up, Scotty.")) # KIRK
print(fast_class_model.predict("Sensor scan to one half parsec. Negative, Captain.")) # SPOCK
print(fast_class_model.predict("Change course, come about to one eight five, mark three.")) # KIRK
print(fast_class_model.predict("I believed the Romulans have developed a cloaking device which renders our tracking sensors useless.")) # SPOCK
print(fast_class_model.predict("Jim, he's dead.")) # MCCOY
print(fast_class_model.predict("Well, you can see for yourself, he's mentally depressed, physically weak, disoriented, displays of feelings of persecution and rebellion.")) # MCCOY

(('__label__KIRK',), array([0.67979342]))
(('__label__SPOCK',), array([0.98693001]))
(('__label__SPOCK',), array([0.48280835]))
(('__label__SPOCK',), array([0.75689733]))
(('__label__MCCOY',), array([0.99999917]))
(('__label__MCCOY',), array([0.66121286]))


2. 词表征

使用`train_unsupervised`方法。

可以指定的**训练参数**有：minn-maxn（设置subgram的最大最小窗口）、dim（词向量维数）、epoch（迭代数）和lr（学习率）

In [8]:
# 训练数据：一行一个句子，每个句子做好预处理
# 可自行调整参数，以下值均为默认值
fast_wv_model = fasttext.train_unsupervised('processed_data/lines_corpus1.txt',
                                           minn=3, maxn=6, dim=100,
                                           epoch=5, lr=0.05)

In [9]:
# print(fast_wv_model.get_word_vector('bridge')) #打印词向量
print("fasttext - most similar words:",fast_wv_model.get_nearest_neighbors('bridge'))

fasttext - most similar words: [(0.9815894365310669, 'ridge'), (0.9703838229179382, 'bailey'), (0.9673263430595398, 'bay'), (0.9645262360572815, 'riley'), (0.9627730250358582, 'sickbay'), (0.9610312581062317, 'tail'), (0.9515268206596375, 'helm'), (0.9505404829978943, 'audio'), (0.9474798440933228, 'tie'), (0.9453719258308411, 'notify')]


不过fasttext的优越性主要体现在分类问题上，尤其对于多类别大样本的数据。